In [49]:
import pandas as pd
df = pd.read_csv("/Users/skylerwilson/Desktop/PartsWise/App/Data/Raw/PartsWiseData.csv")
cols = ['part_number', 'sales_last_jan', 'sales_last_feb', 'sales_last_mar', 'sales_last_apr', 'sales_last_may',
       'sales_last_jun', 'sales_last_jul', 'sales_last_aug', 'sales_last_sep', 'sales_last_oct', 'sales_last_nov', 
       'sales_last_dec', 'sales_jan', 'sales_feb', 'sales_mar', 'sales_apr', 'sales_may', 'sales_jun', 'sales_jul', 
       'sales_aug', 'sales_sep', 'sales_oct', 'sales_nov', 'sales_dec', 'last_received_date', 'last_sold_date']

df[cols]

,part_number,sales_last_jan,sales_last_feb,sales_last_mar,sales_last_apr,sales_last_may,sales_last_jun,sales_last_jul,sales_last_aug,sales_last_sep,...,sales_may,sales_jun,sales_jul,sales_aug,sales_sep,sales_oct,sales_nov,sales_dec,last_received_date,last_sold_date
0,45-5531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,45-8831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,09/11/2014,09/04/2014
2,45-8897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11/05/2015,11/05/2015
3,45-8898,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01/06/2015,05/21/2015
4,45-8985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11/27/2012,11/20/2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71335,45-20090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10/04/2005,NaN
71336,45-2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12/16/2011,02/01/2012
71337,45-2011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12/16/2011,03/09/2012
71338,45-2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12/16/2011,02/09/2013


In [50]:
import pandas as pd
from calendar import monthrange
from datetime import datetime

def update_dates(df):
    """
    Updates 'last_received_date' if it's NaN to "not received".
    For 'last_sold_date', it checks if it's NaN and then:
      - If sold in a particular month either this year or last year, uses the end of that month.
      - Otherwise, it's listed as "not sold".
    It leaves non-NaN 'last_sold_date' values unchanged.
    
    Parameters:
    - df: DataFrame to be processed.
    """
    # Update 'last_received_date' to "not received" if it's NaN
    df['last_received_date'] = df['last_received_date'].fillna("not received")

    # Define current year and last year for reference
    current_year = datetime.now().year
    last_year = current_year - 1

    if pd.isna(df['last_sold_date']).any():
        # Iterate over the DataFrame to update 'last_sold_date' only where it's NaN
        for month in ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']:
            this_year_col = f'sales_{month}'
            last_year_col = f'sales_last_{month}'

            # Check this year's sales
            df.loc[df[this_year_col] > 0, 'last_sold_date'] = df.loc[df[this_year_col] > 0].apply(
                lambda row: datetime(current_year, datetime.strptime(month, '%b').month, monthrange(current_year, datetime.strptime(month, '%b').month)[1]).strftime('%Y-%m-%d') 
                if pd.isna(row['last_sold_date']) else row['last_sold_date'], axis=1)

            # Check last year's sales
            df.loc[df[last_year_col] > 0, 'last_sold_date'] = df.loc[df[last_year_col] > 0].apply(
                lambda row: datetime(last_year, datetime.strptime(month, '%b').month, monthrange(last_year, datetime.strptime(month, '%b').month)[1]).strftime('%Y-%m-%d') 
                if pd.isna(row['last_sold_date']) else row['last_sold_date'], axis=1)
        
        # For any remaining NaN 'last_sold_date', label as "not sold"
        df['last_sold_date'] = df['last_sold_date'].fillna("not sold")

    return df





In [51]:
df = update_dates(df)
df[cols]

,part_number,sales_last_jan,sales_last_feb,sales_last_mar,sales_last_apr,sales_last_may,sales_last_jun,sales_last_jul,sales_last_aug,sales_last_sep,...,sales_may,sales_jun,sales_jul,sales_aug,sales_sep,sales_oct,sales_nov,sales_dec,last_received_date,last_sold_date
0,45-5531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,not received,not sold
1,45-8831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,09/11/2014,09/04/2014
2,45-8897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11/05/2015,11/05/2015
3,45-8898,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01/06/2015,05/21/2015
4,45-8985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11/27/2012,11/20/2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71335,45-20090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10/04/2005,not sold
71336,45-2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12/16/2011,02/01/2012
71337,45-2011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12/16/2011,03/09/2012
71338,45-2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12/16/2011,02/09/2013
